# Playing with Azure Resource Graph (via REST API and APP)
(https://docs.microsoft.com/en-us/rest/api/azure-resourcegraph/)
- Requirement: the APP should have 'Reader' role in the subscription level!

# Required information

In [ ]:
appId = '' 
appSecret = '' 
tenantId = ''
subscription_id = '' 

# Required Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import urllib.parse

# Function to get the token to access via the app

In [ ]:
def getaadtoken(tenantId, appId, appSecret):
    url = "https://login.windows.net/%s/oauth2/token" % (tenantId)
    resourceAppIdUri = 'https://management.azure.com'
    body = {
        'resource' : resourceAppIdUri,
        'client_id' : appId,
        'client_secret' : appSecret,
        'grant_type' : 'client_credentials'
    }
    data = urllib.parse.urlencode(body).encode("utf-8")
    req = urllib.request.Request(url, data)
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    aadToken = jsonResponse["access_token"]
    return aadToken

# Instantiating previous function

In [ ]:
access_token = getaadtoken(tenantId, appId, appSecret)

# Function to get the resources (or run a KQL using the 'resources' table)
https://docs.microsoft.com/en-us/rest/api/azureresourcegraph/resourcegraph(2019-04-01)/resources

In [ ]:
def resourcegraph_resources(access_token,subscriptions_list,kql_query):
    url = "https://management.azure.com/providers/Microsoft.ResourceGraph/resources?api-version=2019-04-01"
    req = urllib.request.Request(url, 
                                 headers={'Authorization' : "Bearer " + access_token, 'Content-Type':'application/json'},
                                 data=bytes(json.dumps({"subscriptions": subscriptions_list, "query": kql_query}), encoding="utf-8"))
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    df_temp = pd.DataFrame.from_records(jsonResponse['data']['rows'])
    df_temp.columns = pd.DataFrame.from_records(jsonResponse['data']['columns'])['name'].values
    return df_temp

# Instantiating previous function

In [ ]:
df = resourcegraph_resources(access_token,[subscription_id],"")

<h1 align='center'>=======================================================<br>=======================================================<br>ANALYSING THE DATA</h1>

# How the data looks like?

In [ ]:
df.head()

# How many resources are in each resource group?

In [ ]:
df['resourceGroup'].value_counts()

# How many resources for each type are there?

In [ ]:
df['type'].value_counts()

# Which resources are in each resource group?

In [ ]:
df.groupby(['resourceGroup'])['type'].apply(list)